# MNIST com Convolução

## Configurando a rede neural

In [1]:
import numpy as np
import keras
from keras import layers

Nesta parte estamaos preparando os dados após importar o dataset **mnist**. A ordem do preprocessamento:
* Divisão dos dados em Teste e Treinamento
* Escalona as imagens entre 0 e 1
  * Os valores dos pixels estão representados em um range de 0 a 255, que é associado entre Preto (0) e Branco (255). Por isso devemos transformamos para float e dpois dividir por 255
* Garantir que cada imagem possui a resolução correta (28x28 pixels)
* Conversão de vetores para matrizes binarias

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 2s 0us/step
x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
60000 train samples
10000 test samples


Aqui estamos constuindo nosso modelo e definindo nossas camadas, além de suas configurações.
Estamos usando 2 camadas de convolução, cada uma com um filtro de 32 e 64, respectivamente, representando a dimensão do output, além de ambas com um tamanho da janela de convolução (kernel_size) de 3x3.
Cada uma dessas camadas é seguida por uma camada de pooling, que reduz o tamanho espacial da representação

## Model

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

Para o treinamento usamos um **batch size** de 128 e 3 **epochs**, como pedido na atividade

## Treinando



In [4]:
batch_size = 128
epochs = 3

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/3
422/422 [==============================] - 13s 6ms/step - loss: 0.3751 - accuracy: 0.8877 - val_loss: 0.0890 - val_accuracy: 0.9768
Epoch 2/3
422/422 [==============================] - 2s 5ms/step - loss: 0.1154 - accuracy: 0.9653 - val_loss: 0.0595 - val_accuracy: 0.9838
Epoch 3/3
422/422 [==============================] - 2s 5ms/step - loss: 0.0862 - accuracy: 0.9731 - val_loss: 0.0450 - val_accuracy: 0.9875


## Verificando o resultado

Abaixo verificamos o resultado e garantimos que tivemos uma acuracia superior a pedida na atividade.

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print(f"Test loss: {score[0]:.4f}")
print(f"Test accuracy: {score[1]:.4f}")

Test loss: 0.0464
Test accuracy: 0.9853
